In [14]:
import tensorflow as tf
import pandas as pd
import os
from sklearn.model_selection import train_test_split

image_size = (224, 224)  
batch_size = 32          

csv_file = "chessmind-rl/data/cells/dataset.csv"  
image_dir = "chessmind-rl/data/cells/"      

df = pd.read_csv(csv_file)

df['image_path'] = df['Filename'].apply(lambda x: os.path.join(image_dir, x))

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

df

,Site,Figure,Color,Filename,image_path
0,chessarena,bishop,black,chessarena_black_bishop_on_black.png,chessmind-rl/data/cells/chessarena_black_bisho...
1,chessarena,bishop,black,chessarena_black_bishop_on_white.png,chessmind-rl/data/cells/chessarena_black_bisho...
2,chessarena,bishop,black,chessarena_black_bishop_selected.png,chessmind-rl/data/cells/chessarena_black_bisho...
3,chessarena,king,black,chessarena_black_king_check.png,chessmind-rl/data/cells/chessarena_black_king_...
4,chessarena,king,black,chessarena_black_king_checkmate.png,chessmind-rl/data/cells/chessarena_black_king_...
...,...,...,...,...,...
169,lichess,queen,white,lichess_white_queen_on_white_selected.png,chessmind-rl/data/cells/lichess_white_queen_on...
170,lichess,rook,white,lichess_white_rook_on_black.png,chessmind-rl/data/cells/lichess_white_rook_on_...
171,lichess,rook,white,lichess_white_rook_on_black_selected.png,chessmind-rl/data/cells/lichess_white_rook_on_...
172,lichess,rook,white,lichess_white_rook_on_white.png,chessmind-rl/data/cells/lichess_white_rook_on_...


In [15]:
def preprocess_image(file_path, label_figure, label_color):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, image_size)
    img = img / 255.0  

    return img, (label_figure, label_color)


def create_dataset(dataframe):
    file_paths = dataframe['image_path'].values
    figure_labels = dataframe['Figure'].map(figure_to_label).values
    color_labels = dataframe['Color'].map(color_to_label).values
    
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, figure_labels, color_labels))
    dataset = dataset.map(lambda x, y, z: preprocess_image(x, y, z), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return dataset


In [16]:
figure_to_label = {figure: idx for idx, figure in enumerate(df['Figure'].unique())}
color_to_label = {color: idx for idx, color in enumerate(df['Color'].unique())}

train_dataset = create_dataset(train_df)
test_dataset = create_dataset(test_df)



In [17]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

def create_custom_model(input_shape, num_figures, num_colors):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    figure_output = layers.Dense(num_figures, activation='softmax', name='figure_output')(x)
    color_output = layers.Dense(num_colors, activation='softmax', name='color_output')(x)
    model = models.Model(inputs=inputs, outputs=[figure_output, color_output])
    return model

input_shape = (224, 224, 3)  
num_figures = len(figure_to_label) 
num_colors = len(color_to_label)   

In [18]:
    
model = create_custom_model(input_shape, num_figures, num_colors)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'figure_output': 'sparse_categorical_crossentropy', 
        'color_output': 'sparse_categorical_crossentropy'
    },
    metrics={
        'figure_output': ['accuracy'],  
        'color_output': ['accuracy']    
    }
)

model.summary()

history = model.fit(train_dataset, epochs=15, validation_data=test_dataset)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 222, 222,  │        896 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 111, 111,  │          0 │ conv2d_8[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 109, 109,  │     18,496 │ max_pooling2d_8[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 54, 54,    │          0 │ conv2d_9[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 52, 52,    │     73,856 │ max_pooling2d_9[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_10    │ (None, 26, 26,    │          0 │ conv2d_10[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 24, 24,    │    147,584 │ max_pooling2d_10… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_11    │ (None, 12, 12,    │          0 │ conv2d_11[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 18432)     │          0 │ max_pooling2d_11… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │  4,718,848 │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ figure_output       │ (None, 8)         │      2,056 │ dropout_2[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ color_output        │ (None, 4)         │      1,028 │ dropout_2[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,962,764 (18.93 MB)

 Trainable params: 4,962,764 (18.93 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 294ms/step - color_output_accuracy: 0.4046 - color_output_loss: 1.4626 - figure_output_accuracy: 0.1474 - figure_output_loss: 2.1866 - loss: 3.6667 - val_color_output_accuracy: 0.9714 - val_color_output_loss: 0.8364 - val_figure_output_accuracy: 0.3143 - val_figure_output_loss: 2.0428 - val_loss: 2.8219
Epoch 2/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 254ms/step - color_output_accuracy: 0.6432 - color_output_loss: 1.0624 - figure_output_accuracy: 0.1487 - figure_output_loss: 2.0765 - loss: 3.1384 - val_color_output_accuracy: 0.8857 - val_color_output_loss: 0.5644 - val_figure_output_accuracy: 0.3714 - val_figure_output_loss: 2.0014 - val_loss: 2.3519
Epoch 3/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step - color_output_accuracy: 0.7623 - color_output_loss: 0.9111 - figure_output_accuracy: 0.2544 - figure_output_loss: 1.9263 - loss: 2.8490 - val_color_output_accuracy: 0.9714 - val_color_output_loss: 0.2660 - val_figure_output_accuracy: 0.3714 - val_figure_output_

In [19]:
from joblib import dump

dump(model, 'chessmind-rl/models/model_figures.joblib')

['chessmind-rl/models/model_figures.joblib']